# LeetCode 97 Interleaving Strings

Given strings `s1`, `s2`, and `s3`, find whether `s3` is formed by an interleaving of `s1` and `s2`.

Method 1: backtracking with memoization

1. Immediately return false for cases where the lengths of the strings do not align
2. Create a function that recursively find if `s3[i+j..]` can be formed by `s1[i..]` and `s2[j..]`
3. Cache the result with `@cache()` or `memo`
   * We can remember only the `false` outcomes since the `true` results are returned directly to the initiating function call.

Method 2: dynamic programming
Similar to the first method, but we can start from the end of the strings. 
We define the problem `dp[i, j] = true` if `s3[i+j..]` can be formed by `s1[i..]` and `s2[j..]`, and `dp[i, j] = false` otherwise.
Hence, we get the following
```py
if s1[i] == s3[i+j] and dp[i+1, j] or s2[j] == s3[i+j] and dp[i+1, j]:
   dp[i, j] = True
else:
   dp[i, j] = False
```

Alternatively, we could assume all strings are reversed and therefore start from index 0.
We therefore need to define the base cases:
1. `dp[0, 0] = true` 
   * the empty strings can match
2. `dp[i, 0] = true for i in range(1, len(s1)+1) if dp[i-1, 0] and s1[i-1] == s3[i-1]` 
   * the first `i` characters of `s1` and `s3` match
3. `dp[0, j] = true for j in range(1, len(s2)+1) if dp[0, j-1] and s2[j-1] == s3[j-1]` 
   * the first `j` characters of `s2` and `s3` match 

Subsequently, we can populate `dp[i, j]` based on whether `dp[i-1, j] and s1[i-1] == s3[i+j-1]` or `dp[i, j-1] and s2[j-1] == s3[i+j-1]` holds true.

In [1]:
class Solution1:
    def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        if len(s1) + len(s2) != len(s3):
            return False
        memo = set()  # only cache the cases that cannot form an interleaving string
        def match(i, j): # i, j: start index for s1, s2, the start index of s3 = i + j
            # terminate condition
            if i + j == len(s3):
                return True
            if (i, j) in memo:
                return False
            if i < len(s1) and s1[i] == s3[i+j] and match(i+1, j):
                return True
            if j < len(s2) and s2[j] == s3[i+j] and match(i, j+1):
                return True
            memo.add((i, j))
            return False
        return match(0, 0)

In [2]:
class Solution2:
    def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        if len(s1) + len(s2) != len(s3):
            return False
        
        dp = [[False for _ in range(len(s2)+1)] for _ in range(len(s1)+1)]
        dp[0][0] = True
        for i in range(len(s1)):
            if s1[i] == s3[i]:
                dp[i+1][0] = True   # only match s1
            else:
                break
        for j in range(len(s2)):
            if s2[j] == s3[j]:
                dp[0][j+1] = True   # only match s2
            else:
                break
        for i in range(1, len(s1)+1):
            for j in range(1, len(s2)+1):
                if s1[i-1] == s3[i+j-1] and dp[i-1][j] or s2[j-1] == s3[i+j-1] and dp[i][j-1]:
                    dp[i][j] = True
        return dp[-1][-1]